In [1]:
#https://github.com/sh0416/bpr/blob/master

In [2]:
# from setuptools import setup, Extension
# from torch.utils import cpp_extension

# setup(name='vsl_cpp',
#       ext_modules=[cpp_extension.CppExtension('vsl_cpp', ['vsl_cpp.cpp'])],
#       cmdclass={'build_ext': cpp_extension.BuildExtension})

In [3]:
#import vsl_cpp
import os
import gzip
import json
import math
import random
import pickle
import pprint
import argparse

import numpy as np
import pandas as pd

from setuptools import setup, Extension

import argparse
from collections import deque

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import cpp_extension
from torch.utils.data import IterableDataset, DataLoader, get_worker_info
from torch.utils.tensorboard import SummaryWriter



In [4]:
class VariableShapeList(object):
    def __init__(self, indexes, data):
        super().__init__()
        self.batch_size = indexes.shape[0] - 1
        self.indexes = indexes
        self.data = data

    @classmethod
    def from_tensors(cls, tensors):
        """
        Args:
            tensors (list(torch.LongTensor)): list of tensors which have different shape.
                                              For now, operation is only supported for one dimensional tensor.
        """
        assert len(tensors) > 0, "`tensors` is empty"
        assert all([len(x.shape) == 1 for x in tensors]), "Some elements in `tensors` are not one dimensional"
        # Initialize `batch_size`
        batch_size = len(tensors)
        
        # Build up `indexes`
        indexes = torch.empty((batch_size+1,), dtype=torch.long)
        indexes[0], tmp = 0, 0
        for idx, tensor in enumerate(tensors, start=1):
            tmp += tensor.numel()
            indexes[idx] = tmp
        
        # Build up `data`
        data = torch.empty((indexes[batch_size],), dtype=torch.long)
        for idx, tensor in enumerate(tensors, start=0):
            data[indexes[idx]:indexes[idx+1]] = tensor
        return cls(indexes, data)

    def __getitem__(self, idx):
        return self.data[self.indexes[idx]:self.indexes[idx+1]]
    
    def get_size_tensor(self):
        return self.indexes[1:] - self.indexes[:-1]


def vsl_intersection(list1, list2):
    """Calculate intersection VariableShapeList.
    For the first, we doesn't care about the duplicated item in the list.
    So, you must check whether the element in the list is not duplicated.
    
    Args:
        list1 (VariableShapeList): 
        list2 (VariableShapeList):
        
    Returns:
        VariableShapeList
    """
    # get `batch_size`
    assert list1.batch_size == list2.batch_size, "list1 and list2 have different batch size"
    data, indexes = vsl_cpp.vsl_intersection(list1.data,
                                             list1.indexes,
                                             list2.data,
                                             list2.indexes)
    return VariableShapeList(indexes, data)
    

def vsl_precision(pred, true):
    """Calculate precision.
    
    Args:
        pred (VariableShapeList): 
        true (variableShapeList): 
    Returns:
        torch.FloatTensor [batch_size]
    """
    if not torch.all(pred.get_size_tensor()>0):
        raise ZeroDivisionError("The denominator of precision could be zero")
    intersection = vsl_intersection(pred, true)
    intersection_size = intersection.get_size_tensor().float()
    pred_size = pred.get_size_tensor().float()
    return intersection_size / pred_size
    
    
def vsl_recall(pred, true):
    """Calculate recall.
    
    Args:
        pred (VariableShapeList): 
        true (variableShapeList): 
    Returns:
        torch.FloatTensor [batch_size]
    """
    if not torch.all(true.get_size_tensor()>0):
        raise ZeroDivisionError("The denominator of precision could be zero")
    intersection = vsl_intersection(pred, true)
    intersection_size = intersection.get_size_tensor().float()
    true_size = true.get_size_tensor().float()
    return intersection_size / true_size
    
    
def precision_and_recall_k(user_emb, item_emb, train_user_list, test_user_list, klist, batch=512):
    """Compute precision at k using GPU.

    Args:
        user_emb (torch.Tensor): embedding for user [user_num, dim]
        item_emb (torch.Tensor): embedding for item [item_num, dim]
        train_user_list (list(set)):
        test_user_list (list(set)):
        k (list(int)):
    Returns:
        (torch.Tensor, torch.Tensor) Precision and recall at k
    """
    # Calculate max k value
    max_k = max(klist)

    # Compute all pair of training and test record
    result = None
    for i in range(0, user_emb.shape[0], batch):
        # Create already observed mask
        mask = user_emb.new_ones([min([batch, user_emb.shape[0]-i]), item_emb.shape[0]])
        for j in range(batch):
            if i+j >= user_emb.shape[0]:
                break
            mask[j].scatter_(dim=0, index=torch.tensor(list(train_user_list[i+j])).cuda(), value=torch.tensor(0.0).cuda())
        # Calculate prediction value
        cur_result = torch.mm(user_emb[i:i+min(batch, user_emb.shape[0]-i), :], item_emb.t())
        cur_result = torch.sigmoid(cur_result)
        assert not torch.any(torch.isnan(cur_result))
        # Make zero for already observed item
        cur_result = torch.mul(mask, cur_result)
        _, cur_result = torch.topk(cur_result, k=max_k, dim=1)
        result = cur_result if result is None else torch.cat((result, cur_result), dim=0)

    result = result.cpu()
    # Sort indice and get test_pred_topk
    precisions, recalls = [], []
    for k in klist:
        precision, recall = 0, 0
        for i in range(user_emb.shape[0]):
            test = test_user_list[i]
            pred = set(result[i, :k].numpy().tolist())
            val = len(test & pred)
            precision += val / min([k, len(test)])
            recall += val / len(test)
        precisions.append(precision / user_emb.shape[0])
        recalls.append(recall / user_emb.shape[0])
    return precisions, recalls


def accuracy(pred, true, total_size):
    """Calculate accuracy.
    
    Args:
        pred (VariableLengthList): 
        true (VariableLengthList): 
        total_size (torch.FloatTensor): [batch_size] or scalar tensor when the `total_size` is equal over all examples.
    Returns:
        torch.FloatTensor [batch_size]
    """
    raise NotImplemented
    

def mean_average_precision(pred, true):
    raise NotImplemented
    

In [5]:
class DatasetLoader(object):
    def load(self):
        """Minimum condition for dataset:
          * All users must have at least one item record.
          * All items must have at least one user record.
        """
        raise NotImplementedError

class MovieLens20M(DatasetLoader):
    def __init__(self, data_dir):
        self.fpath = os.path.join(data_dir, 'test_set.csv')

    def load(self):
        df = pd.read_csv(self.fpath,
                         sep=',',
                         names=['user', 'item', 'rate', 'time'],
                         usecols=['user', 'item', 'time'],
                         skiprows=1)
        return df

def convert_unique_idx(df, column_name):
    column_dict = {x: i for i, x in enumerate(df[column_name].unique())}
    df[column_name] = df[column_name].apply(column_dict.get)
    df[column_name] = df[column_name].astype('int')
    assert df[column_name].min() == 0
    assert df[column_name].max() == len(column_dict) - 1
    return df, column_dict


def create_user_list(df, user_size):
    user_list = [list() for u in range(user_size)]
    for row in df.itertuples():
        user_list[row.user].append((row.time, row.item))
    return user_list


def split_train_test(df, user_size, test_size=0.2, time_order=False):
    """Split a dataset into `train_user_list` and `test_user_list`.
    Because it needs `user_list` for splitting dataset as `time_order` is set,
    Returning `user_list` data structure will be a good choice."""
    # TODO: Handle duplicated items
    if not time_order:
        test_idx = np.random.choice(len(df), size=int(len(df)*test_size))
        train_idx = list(set(range(len(df))) - set(test_idx))
        test_df = df.loc[test_idx].reset_index(drop=True)
        train_df = df.loc[train_idx].reset_index(drop=True)
        test_user_list = create_user_list(test_df, user_size)
        train_user_list = create_user_list(train_df, user_size)
    else:
        total_user_list = create_user_list(df, user_size)
        train_user_list = [None] * len(user_list)
        test_user_list = [None] * len(user_list)
        for user, item_list in enumerate(total_user_list):
            # Choose latest item
            item_list = sorted(item_list, key=lambda x: x[0])
            # Split item
            test_item = item_list[math.ceil(len(item_list)*(1-test_size)):]
            train_item = item_list[:math.ceil(len(item_list)*(1-test_size))]
            # Register to each user list
            test_user_list[user] = test_item
            train_user_list[user] = train_item
    # Remove time
    test_user_list = [list(map(lambda x: x[1], l)) for l in test_user_list]
    train_user_list = [list(map(lambda x: x[1], l)) for l in train_user_list]
    return train_user_list, test_user_list


def create_pair(user_list):
    pair = []
    for user, item_list in enumerate(user_list):
        pair.extend([(user, item) for item in item_list])
    return pair


def main(args):
    df = MovieLens20M(args.data_dir).load()
    df, user_mapping = convert_unique_idx(df, 'user')
    df, item_mapping = convert_unique_idx(df, 'item')
    print('Complete assigning unique index to user and item')

    user_size = len(df['user'].unique())
    item_size = len(df['item'].unique())

    train_user_list, test_user_list = split_train_test(df,
                                                       user_size,
                                                       test_size=args.test_size,
                                                       time_order=args.time_order)
    print('Complete spliting items for training and testing')

    train_pair = create_pair(train_user_list)
    print('Complete creating pair')

    dataset = {'user_size': user_size, 'item_size': item_size, 
               'user_mapping': user_mapping, 'item_mapping': item_mapping,
               'train_user_list': train_user_list, 'test_user_list': test_user_list,
               'train_pair': train_pair}
    dirname = os.path.dirname(os.path.abspath(args.output_data))
    os.makedirs(dirname, exist_ok=True)
    with open(args.output_data, 'wb') as f:
        pickle.dump(dataset, f, protocol=pickle.HIGHEST_PROTOCOL)


if __name__ == '__main__':
    # Parse argument
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--dataset',
    #                     choices=['test_set'],
    #                     default='test_set',
    #                     help="Dataset to use")
    # parser.add_argument('--data_dir',
    #                     type=str,
    #                     default=os.path.join('test_set'),
    #                     help="File path for raw data")
    # parser.add_argument('--output_data',
    #                     type=str,
    #                     default=os.path.join('preprocessed', 'test_set.pickle'),
    #                     help="File path for preprocessed data")
    # parser.add_argument('--test_size',
    #                     type=float,
    #                     default=0.2,
    #                     help="Proportion for training and testing split")
    # parser.add_argument('--time_order',
    #                     action='store_true',
    #                     help="Use time order for splitting training and testing data")
    # args = parser.parse_args()
    args = argparse.Namespace()
    args.dataset = 'test_set'
    args.data_dir = os.path.join('data')
    args.output_data = os.path.join('preprocessed', 'test_set.pickle')
    args.test_size = 0.2
    args.time_order = False
    main(args)


Complete assigning unique index to user and item
Complete spliting items for training and testing
Complete creating pair


In [6]:
# setup(name='vsl_op',
#       ext_modules=[cpp_extension.CppExtension('vsl_op', ['vsl_op.cpp'])],
#       cmdclass={'build_ext': cpp_extension.BuildExtension})

In [7]:
class TripletUniformPair(IterableDataset):
    def __init__(self, num_item, user_list, pair, shuffle, num_epochs):
        self.num_item = num_item
        self.user_list = user_list
        self.pair = pair
        self.shuffle = shuffle
        self.num_epochs = num_epochs

    def __iter__(self):
        worker_info = get_worker_info()
        # Shuffle per epoch
        self.example_size = self.num_epochs * len(self.pair)
        self.example_index_queue = deque([])
        self.seed = 0
        if worker_info is not None:
            self.start_list_index = worker_info.id
            self.num_workers = worker_info.num_workers
            self.index = worker_info.id
        else:
            self.start_list_index = None
            self.num_workers = 1
            self.index = 0
        return self

    def __next__(self):
        if self.index >= self.example_size:
            raise StopIteration
        # If `example_index_queue` is used up, replenish this list.
        while len(self.example_index_queue) == 0:
            index_list = list(range(len(self.pair)))
            if self.shuffle:
                random.Random(self.seed).shuffle(index_list)
                self.seed += 1
            if self.start_list_index is not None:
                index_list = index_list[self.start_list_index::self.num_workers]
                # Calculate next start index
                self.start_list_index = (self.start_list_index + (self.num_workers - (len(self.pair) % self.num_workers))) % self.num_workers
            self.example_index_queue.extend(index_list)
        result = self._example(self.example_index_queue.popleft())
        self.index += self.num_workers
        return result

    def _example(self, idx):
        u = self.pair[idx][0]
        i = self.pair[idx][1]
        j = np.random.randint(self.num_item)
        while j in self.user_list[u]:
            j = np.random.randint(self.num_item)
        return u, i, j


class BPR(nn.Module):
    def __init__(self, user_size, item_size, dim, weight_decay):
        super().__init__()
        self.W = nn.Parameter(torch.empty(user_size, dim))
        self.H = nn.Parameter(torch.empty(item_size, dim))
        nn.init.xavier_normal_(self.W.data)
        nn.init.xavier_normal_(self.H.data)
        self.weight_decay = weight_decay

    def forward(self, u, i, j):
        """Return loss value.
        
        Args:
            u(torch.LongTensor): tensor stored user indexes. [batch_size,]
            i(torch.LongTensor): tensor stored item indexes which is prefered by user. [batch_size,]
            j(torch.LongTensor): tensor stored item indexes which is not prefered by user. [batch_size,]
        
        Returns:
            torch.FloatTensor
        """
        u = self.W[u, :]
        i = self.H[i, :]
        j = self.H[j, :]
        x_ui = torch.mul(u, i).sum(dim=1)
        x_uj = torch.mul(u, j).sum(dim=1)
        x_uij = x_ui - x_uj
        log_prob = F.logsigmoid(x_uij).sum()
        regularization = self.weight_decay * (u.norm(dim=1).pow(2).sum() + i.norm(dim=1).pow(2).sum() + j.norm(dim=1).pow(2).sum())
        return -log_prob + regularization

    def recommend(self, u):
        """Return recommended item list given users.

        Args:
            u(torch.LongTensor): tensor stored user indexes. [batch_size,]

        Returns:
            pred(torch.LongTensor): recommended item list sorted by preference. [batch_size, item_size]
        """
        u = self.W[u, :]
        x_ui = torch.mm(u, self.H.t())
        pred = torch.argsort(x_ui, dim=1)
        return pred


def precision_and_recall_k(user_emb, item_emb, train_user_list, test_user_list, klist, batch=512):
    """Compute precision at k using GPU.

    Args:
        user_emb (torch.Tensor): embedding for user [user_num, dim]
        item_emb (torch.Tensor): embedding for item [item_num, dim]
        train_user_list (list(set)):
        test_user_list (list(set)):
        k (list(int)):
    Returns:
        (torch.Tensor, torch.Tensor) Precision and recall at k
    """
    # Calculate max k value
    max_k = max(klist)

    # Compute all pair of training and test record
    result = None
    for i in range(0, user_emb.shape[0], batch):
        # Create already observed mask
        mask = user_emb.new_ones([min([batch, user_emb.shape[0]-i]), item_emb.shape[0]])
        for j in range(batch):
            if i+j >= user_emb.shape[0]:
                break
            mask[j].scatter_(dim=0, index=torch.tensor(list(train_user_list[i+j])).cuda(), value=torch.tensor(0.0).cuda())
        # Calculate prediction value
        cur_result = torch.mm(user_emb[i:i+min(batch, user_emb.shape[0]-i), :], item_emb.t())
        cur_result = torch.sigmoid(cur_result)
        assert not torch.any(torch.isnan(cur_result))
        # Make zero for already observed item
        cur_result = torch.mul(mask, cur_result)
        _, cur_result = torch.topk(cur_result, k=max_k, dim=1)
        result = cur_result if result is None else torch.cat((result, cur_result), dim=0)

    result = result.cpu()
    # Sort indice and get test_pred_topk
    precisions, recalls = [], []
    for k in klist:
        precision, recall = 0, 0
        for i in range(user_emb.shape[0]):
            test = set(test_user_list[i])
            pred = set(result[i, :k].numpy().tolist())
            val = len(test & pred)
            precision += val / max([min([k, len(test)]), 1])
            recall += val / max([len(test), 1])
        precisions.append(precision / user_emb.shape[0])
        recalls.append(recall / user_emb.shape[0])
    return precisions, recalls


def main(args):
    # Initialize seed
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

    # Load preprocess data
    with open(args.data, 'rb') as f:
        dataset = pickle.load(f)
        user_size, item_size = dataset['user_size'], dataset['item_size']
        train_user_list, test_user_list = dataset['train_user_list'], dataset['test_user_list']
        train_pair = dataset['train_pair']
    print('Load complete')

    # Create dataset, model, optimizer
    dataset = TripletUniformPair(item_size, train_user_list, train_pair, True, args.n_epochs)
    loader = DataLoader(dataset, batch_size=args.batch_size, num_workers=16)
    model = BPR(user_size, item_size, args.dim, args.weight_decay).cuda()
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    writer = SummaryWriter()

    # Training
    smooth_loss = 0
    idx = 0
    for u, i, j in loader:
        optimizer.zero_grad()
        loss = model(u, i, j)
        loss.backward()
        optimizer.step()
        writer.add_scalar('train/loss', loss, idx)
        smooth_loss = smooth_loss*0.99 + loss*0.01
        if idx % args.print_every == (args.print_every - 1):
            print('loss: %.4f' % smooth_loss)
        if idx % args.eval_every == (args.eval_every - 1):
            plist, rlist = precision_and_recall_k(model.W.detach(),
                                                    model.H.detach(),
                                                    train_user_list,
                                                    test_user_list,
                                                    klist=[1, 5, 10])
            print('P@1: %.4f, P@5: %.4f P@10: %.4f, R@1: %.4f, R@5: %.4f, R@10: %.4f' % (plist[0], plist[1], plist[2], rlist[0], rlist[1], rlist[2]))
            writer.add_scalars('eval', {'P@1': plist[0],
                                                    'P@5': plist[1],
                                                    'P@10': plist[2]}, idx)
            writer.add_scalars('eval', {'R@1': rlist[0],
                                                'R@5': rlist[1],
                                                'R@10': rlist[2]}, idx)
        if idx % args.save_every == (args.save_every - 1):
            dirname = os.path.dirname(os.path.abspath(args.model))
            os.makedirs(dirname, exist_ok=True)
            torch.save(model.state_dict(), args.model)
        idx += 1


if __name__ == '__main__':
    # Parse argument
    args = argparse.Namespace(
    data=os.path.join('preprocessed', 'ml-1m.pickle'),
    seed=0,
    dim=4,
    lr=1e-3,
    weight_decay=0.025,
    n_epochs=500,
    batch_size=4096,
    print_every=20,
    eval_every=1000,
    save_every=10000,
    model=os.path.join('output', 'bpr.pt')
)
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--data',
    #                     type=str,
    #                     default=os.path.join('preprocessed', 'ml-1m.pickle'),
    #                     help="File path for data")
    # # Seed
    # parser.add_argument('--seed',
    #                     type=int,
    #                     default=0,
    #                     help="Seed (For reproducability)")
    # # Model
    # parser.add_argument('--dim',
    #                     type=int,
    #                     default=4,
    #                     help="Dimension for embedding")
    # # Optimizer
    # parser.add_argument('--lr',
    #                     type=float,
    #                     default=1e-3,
    #                     help="Learning rate")
    # parser.add_argument('--weight_decay',
    #                     type=float,
    #                     default=0.025,
    #                     help="Weight decay factor")
    # # Training
    # parser.add_argument('--n_epochs',
    #                     type=int,
    #                     default=500,
    #                     help="Number of epoch during training")
    # parser.add_argument('--batch_size',
    #                     type=int,
    #                     default=4096,
    #                     help="Batch size in one iteration")
    # parser.add_argument('--print_every',
    #                     type=int,
    #                     default=20,
    #                     help="Period for printing smoothing loss during training")
    # parser.add_argument('--eval_every',
    #                     type=int,
    #                     default=1000,
    #                     help="Period for evaluating precision and recall during training")
    # parser.add_argument('--save_every',
    #                     type=int,
    #                     default=10000,
    #                     help="Period for saving model during training")
    # parser.add_argument('--model',
    #                     type=str,
    #                     default=os.path.join('output', 'bpr.pt'),
    #                     help="File path for model")
    # args = parser.parse_args()
    main(args)

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessed\\ml-1m.pickle'